<a href="https://colab.research.google.com/github/godpeny/laboratory/blob/master/Study/NLP_Using_Deep_Learning/Bidirectional_Encoder_Representation_From_Transformer/machine_reading_comprehension_using_bert_ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Default Setting For Using TPU in Google Colab

In [2]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

In [3]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf
import keras

In [4]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2023-12-28 02:15:02--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   167MB/s    in 0.2s    

2023-12-28 02:15:03 (167 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2023-12-28 02:15:03--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [5]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [6]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [7]:
print(train_contexts[0])

1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [8]:
print(train_questions[0])
print(train_answers[0])
print(list(train_contexts[0])[54], list(train_contexts[0])[55], list(train_contexts[0])[56])

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
{'text': '교향곡', 'answer_start': 54}
교 향 곡


# Data Preprocessing

In [9]:
def add_end_index(contexts, answers):
    for context, answer in zip(contexts, answers):
        gold_text = answer['text']
        gold_text = gold_text.rstrip() # 오른쪽 공백 제거

        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # validation
        assert context[start_idx:end_idx] == gold_text, '{} != {}'.format(context[start_idx:end_idx], gold_text)

        answer['answer_end'] = end_idx

add_end_index(train_contexts, train_answers)
add_end_index(val_contexts, val_answers)

In [10]:
print(train_answers[0])

{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


# Tokenizing

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

# tokenizer : context와 question을 합쳐서 하나의 문장으로 만들어줌
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)
# check
print(train_encodings[0].tokens) # [CLS] 본문 토큰화 결과 [SEP] 질문 토큰화 결과 [SEP] [PAD] ... [PAD]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '.', '여기', '##의', '

In [12]:
# check
print(train_encodings.char_to_token(0,54)) # 54번째 글자의 토큰 인덱스 = 26
print(train_encodings[0].tokens[26]) # 교향곡
print(train_encodings.char_to_token(0,56)) # 56번째 글자의 토큰 인덱스 = 26
print(train_encodings[0].tokens[26]) # 교향곡
print(train_encodings[0].ids[26]) # 교향곡 = 26번째 id = 19282
print(tokenizer.convert_ids_to_tokens(19282)) # 19282 = 교향곡

26
교향곡
26
교향곡
19282
교향곡


In [13]:
def add_token_positions(encodings, answers):
    start_pos = []
    end_pos = []
    deleting_list = []

    for i in range(len(answers)):
        start = encodings.char_to_token(i, answers[i]['answer_start']) # i th sequence에서 answer_start에 해당하는 토큰 인덱스
        end = encodings.char_to_token(i, answers[i]['answer_end'] - 1) # since 'answer_end' is exclusive, so we need to decrease by 1

        if start is not None and end is not None:
            start_pos.append(start)
            end_pos.append(end)
        else:
            start_pos.append(tokenizer.model_max_length)
            end_pos.append(tokenizer.model_max_length)
            deleting_list.append(i)

    encodings.update({'start_positions': start_pos, 'end_positions': end_pos}) # adding start and end positions to `encodings.keys()`
    return deleting_list

In [14]:
train_deleting_list = add_token_positions(train_encodings, train_answers)
test_deleting_list = add_token_positions(val_encodings, val_answers)

print(train_encodings.keys()) # 'start_positions', 'end_positions' are added.

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [15]:
print(train_deleting_list)
print(test_deleting_list)

[711, 726, 728, 729, 761, 765, 767, 768, 805, 2586, 2587, 2722, 2724, 2725, 2731, 3392, 3475, 3478, 3491, 3495, 3498, 3919, 4462, 4465, 4513, 4515, 4565, 4765, 4766, 4772, 4774, 4779, 5334, 6603, 6638, 6639, 6748, 6749, 6750, 6765, 6766, 6771, 6776, 6897, 6898, 6900, 7739, 7741, 9203, 9211, 10880, 11039, 11212, 11727, 11776, 11788, 11789, 11791, 12168, 13708, 13711, 13996, 14460, 14461, 14491, 14724, 14729, 14885, 15764, 15970, 15971, 15973, 15974, 15976, 15977, 15979, 15980, 16080, 17683, 17815, 17828, 18389, 18392, 19045, 19052, 19053, 19195, 19636, 19637, 19638, 19640, 19656, 19761, 19764, 19765, 20614, 20618, 21224, 21243, 21334, 21335, 21338, 21361, 21521, 21522, 22627, 22633, 24003, 24577, 24579, 24580, 24768, 25108, 25176, 25182, 25185, 25186, 25187, 25188, 25448, 25451, 25454, 25457, 25460, 27105, 27112, 27113, 27114, 27159, 27293, 27295, 27555, 27558, 28025, 28438, 28779, 29162, 29189, 29289, 29290, 29855, 31889, 31890, 31891, 31894, 31905, 32050, 32051, 32057, 32058, 32059, 3

In [16]:
def filter(encodings, deleting_list):
    input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
    attention_mask = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
    start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
    end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

    X_data = [input_ids, attention_mask]
    y_data = [start_positions, end_positions]

    return X_data, y_data

In [17]:
X_train, y_train = filter(train_encodings, train_deleting_list)
X_test, y_test = filter(val_encodings, test_deleting_list)

print(len(X_train[0]), len(y_train[0]))
print(len(X_test[0]), len(y_test[0]))

60140 60140
5717 5717


# Modeling

In [18]:
from transformers import TFBertModel

In [32]:
class TFBertForQuestionAnswering(keras.models.Model):
    def __init__(self):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained('klue/bert-base')
        self.qa_output = keras.layers.Dense(units=2, kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02), name='qa_outputs')
        self.softmax = keras.layers.Activation(keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        output = self.bert(input_ids, attention_mask=attention_mask)

        logits = self.qa_output(output[0]) # use all sequence outputs
        #print(logits.shape) # (batch_size, sequence_length=512, 2) last dimension is 2 because 'qa_outputs' has 2 units
        start_logits, end_logits = tf.split(logits, axis=-1, num_or_size_splits=2) # split into start and end logits
        #print(start_logits.shape, end_logits.shape) # (batch, 512, 1)

        start_logits = tf.squeeze(start_logits, axis=-1) # remove dimension of size 1
        end_logits = tf.squeeze(end_logits, axis=-1)

        #print(start_logits.shape, end_logits.shape) # (batch, 512)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [33]:
with strategy.scope():
  model = TFBertForQuestionAnswering()
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already u

In [34]:
history = model.fit(
    X_train,
    y_train,
    epochs=1,
    verbose=1,
    batch_size=16,
)

3759/3759 [==============================] - 527s 115ms/step - loss: 1.1256 - output_1_loss: 0.5070 - output_2_loss: 0.6186


# Prediction

In [26]:
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)

  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1] # X_test[0][idx][start_position ~ end_position]
  print('정답 :',tokenizer.decode(answer_encoded))

  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]]) # input_ids, attention_mask
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]

  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [35]:
predict_test_data_by_idx(0)

본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답 : 1989년 2월 15일
예측 : 1989년 2월 15일
----------------------------------------
